<a href="https://colab.research.google.com/github/Sameer-Arora/CSL203Marathon1/blob/master/BTP_Meta_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [56]:
from google.colab import drive
# drive.mount('/content/drive')
 
# import os  
# os.chdir("/content/drive/My Drive/BTP/Codes/learning-to-learn-by-pytorch-master/")

import argparse  
from __future__ import absolute_import, division, print_function, unicode_literals

# Install TensorFlow
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf
import numpy as np
import os

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten,Softmax,ReLU,Conv2D,Activation,MaxPool2D,LSTMCell,RNN

# Load the TensorBoard notebook extension
%load_ext tensorboard
LOGDIR = "sample_data/"
os.chdir(LOGDIR)
print(os.getcwd())

train_summary_writer = tf.summary.create_file_writer(LOGDIR)

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
/content/sample_data/sample_data/sample_data


In [0]:
def build_meta_model(n_hidden,n_layers,n_trainable_vars):
    cell_list = []
    lstm_cell1 = LSTMCell(n_hidden)
    lstm_cell2 = LSTMCell(n_hidden)
    lstm_cell3 = LSTMCell(1)
    for i in range(n_trainable_vars):
        cell_list.append( tf.keras.layers.StackedRNNCells([lstm_cell1]+[lstm_cell2 for _ in range(n_layers-2)] +[lstm_cell3] ) ) # n_layers = 2 according to the paper.
#         cell_list.append( tf.keras.layers.StackedRNNCells([lstm_cell2 for _ in range(n_layers)] ) ) # n_layers = 2 according to the paper.
    return cell_list

In [0]:
def build_model(out_dim):
  model= Sequential()
  model.add(Dense(out_dim+2,dynamic=True))
  model.add(Dense(out_dim,dynamic=True))
  return model  
  

In [0]:
def get_num_parameters(vars):
  total_vars=0
  for count,var in enumerate(vars):
#     print(grad_var)
    total_vars += tf.reshape(var,[-1]).shape[0]
    print(total_vars)
  
  return total_vars
#   for grad,var in grads_vars:
    
def reshape_params(grads_vars_):
  print(grads_vars_)
#   print(grads_vars_[0][0].shape)
#   print(grads_vars_[0][1].shape)
  grads_vars=[[],[]]
  
  for count,grad_var in enumerate(grads_vars_):
#   for grad_var in grads_vars_:
#     print(grad_var)
    grad=grad_var[0]
    var= grad_var[1]
#     print(grad.shape)
#     print(var.shape)
    grads_vars[0]+= tf.unstack(tf.reshape(grad,[-1]))
    grads_vars[1]+= tf.unstack( tf.reshape(var,[-1]) )
    print(grads_vars[0])
    print(grads_vars[1])
  return zip(grads_vars[0],grads_vars[1])  

In [0]:

def get_gradients(model,input,label):
  with tf.GradientTape() as tape:
    loss= compute_loss(model,input,label)
    print(model.trainable_variables)
  
  grads_vars=[]  
#   for grad,var in tape.gradient(loss,model.trainable_variables):
#     grads_vars.append([tf.reshape(grad,[-1]) , tf.reshape(var,[-1])])
#   return grads_vars
  return tape.gradient(loss,model.trainable_variables)

In [0]:
# def get_meta_gradients(loss, tvars):
#   with tf.GradientTape() as tape:
#     return tape.gradient(loss, tvars)

In [0]:
def get_meta_model_trainable_parameters(cell_list):
  var = cell_list.trainable_variables
  '''
  for cell in cell_list :
     var += cell.trainable_variables
  '''    
  #print(var)
  return var

In [0]:
def compute_loss(model,input,label):
  return tf.reduce_sum(model(input) - label)

In [0]:

def update_model_params(g_new_vars):
  for count,grad_var in enumerate(g_new_vars):
    grad=grad_var[0]
    var= grad_var[1]
    var = var-learning_rate*grad 
    


In [0]:
## trying out stacked LSTM cells

n_samplings=10
n_unroll=5
n_dimension=1
n_hidden=5
n_layers=2



In [0]:

# @tf.function
def trainOptimizer(n_samplings,n_unroll,n_dimension,n_hidden,n_layers,batch_size):
    """
    params:
    n_samplings: number of random function samplings. L will be averaged over T.
    n_unroll: the trained optimizers were unrolled for 20 steps.
    n_dimension: the dimension of input data space.
    n_hidden: This will be used when we actually use this rnn to generate a search direction for the next time step.
    n_layers: n_layer LSTM architecture.

    """
    
    loss = 0
    inp_dim = [batch_size,n_dimension]
    out_dim = n_dimension
    
    model= build_model(out_dim)
    model.build(inp_dim)
    n_trainable_vars = get_num_parameters(model.trainable_variables)
    print("n_trainable_vars= ",n_trainable_vars)
    
    for t in range(n_samplings):
        with tf.GradientTape(persistent=True) as tape:

          cell_list = build_meta_model(n_hidden,n_layers,n_trainable_vars)
          # random sampling of one instance of the quadratic function
          batch_size = 1
  #         state_list = [cell_list[i].zero_state(batch_size, tf.float32) for i in range(n_dimension)] 
          sum_f = 0
          g_new_list = []
          model= build_model(n_dimension)
          y = tf.random.normal([n_dimension, 1])
          x = tf.random.normal([n_dimension, 1])
          grad_f = get_gradients(model,x,y)
          grad_f= reshape_params(zip(grad_f,model.trainable_variables))

          state_list = [cell_list[i].get_initial_state(batch_size=batch_size, dtype=tf.float32) for i in range(n_trainable_vars)] 
          new_grad_f=[]
          new_grad_var=[]
          loss = 0
          for _ in range(n_unroll):
            for i,grad_var in enumerate(grad_f):
                cell = cell_list[i]; state = state_list[i]
                grad_h_t = grad_var[0]        ## verify for the models.
                #print("grad_h_t,\n",grad_h_t.shape,"state\n",state)
                grad_h_t =  tf.expand_dims(tf.expand_dims(grad_h_t,axis=0),axis=1)
                #print("grad_h_t,\n",grad_h_t.shape,"state\n",state)


                #Inputs to different lstm are of different shapes so same Lstm model cannot be used
                cell_output, state = cell(grad_h_t , state) # g_new should be a scalar b/c grad_h_t is a scalar
                #print(cell_output, state )

    #                 softmax_w = tf.Variable(tf.zeros([n_hidden,1]),name="softmax_w")
    #                 softmax_b = tf.Variable(tf.zeros([1]),name="softmax_b")
                g_new_i = cell_output
#                 tape.watch(grad_var[1])
                new_grad_f.append(g_new_i)
                new_grad_var.append(grad_var[1])
                state_list[i] = state # for the next t

            #new_grad_var += new_grad_f
            
#             update_model_params(zip(new_grad_f,new_grad_var),model)
            for count,grad in enumerate(new_grad_f):
              #grad=grad_var[0]
              #var= grad_var[1]
              #print("This is Truth",new_grad_var[0])
              new_grad_var[count] += grad 
            
            count=0
            
#             theta = tf.add(theta, g_new)
#             for layer in model.layers:
#               print("Old",layer.get_weights())
      
            weights_update=[]   
            last_used = 0
            for i in range(len(model.layers)):
                # check to make sure only conv and fully connected layers are assigned weights.
                if 'conv' in model.layers[i].name or 'out' in model.layers[i].name or 'dense' in model.layers[i].name:
                    weights_shape = model.layers[i].kernel.shape
                    no_of_weights = tf.reduce_prod(weights_shape)
                    new_weights = tf.reshape( tf.convert_to_tensor( new_grad_var[count:no_of_weights ] ), weights_shape)
                    model.layers[i].kernel = new_weights
                    last_used += no_of_weights

                if model.layers[i].use_bias:
                    weights_shape = model.layers[i].bias.shape
                    no_of_weights = tf.reduce_prod(weights_shape)
                    new_weights = tf.reshape(tf.convert_to_tensor( new_grad_var[count:count+no_of_weights ] ), weights_shape)
                    model.layers[i].bias = new_weights
                    last_used += no_of_weights
                  
#               for weights in layer.trainable_variables:
#                 print("++++++++++++++++\n",weights)

#                 weights_update.append( tf.reshape(tf.convert_to_tensor( new_grad_var[count:count+np.product(weights.shape) ] ) ,weights.shape ) ) 
#                 weights = tf.reshape(tf.convert_to_tensor( new_grad_var[count:count+np.product(weights.shape) ] ) ,weights.shape ) 
#                 count+=np.product(weights.shape)
              
#               layer.set_weights(  np.array(weights_update) ) 
            
#               print("Old",layer.get_weights())
              
            for layer in model.layers:
#               print("New",layer.get_weights())
                pass
            loss += compute_loss(model,x,y)
            #loss += tf.reduce_sum(model(x) - y)
          print(loss)
          loss = loss / n_unroll    
            
      # update parameter 
  #         g_new = tf.reshape(tf.squeeze(tf.stack(g_new_list)), [n_dimension, 1]) # should be a [n_dimension, 1] tensor
          #print(new_grad_f)

          #sum_f = sum_f + f_at_theta_t 

          #loss += sum_
#           print( "watched vars", tape.watched_variables() )

          #tvars = get_meta_model_trainable_parameters(cell_list[0])
          tvars = cell_list[0].trainable_variables
          #tvars = tf.trainable_variables() # should be just the variable inside the RNN
#           grads = tape.gradient(new_grad_var, tvars)
          grads = tape.gradient(loss, tvars)
          print("----------------\n",grads,"----------------\n")
#           lr = 0.001 # Technically I need to do random search to decide this
          optimizer = tf.keras.optimizers.Adam()
          train_op = optimizer.apply_gradients(zip(grads, tvars))



In [75]:
n_samplings=1
n_unroll=1
n_dimension=3
n_hidden=5
n_layers=2
batch_size=1
learning_rate=0.001  ## meta_learning_rate     

trainOptimizer(n_samplings,n_unroll,n_dimension,n_hidden,n_layers,batch_size)

15
20
35
38
n_trainable_vars=  38
[<tf.Variable 'sequential_23/dense_45/kernel:0' shape=(1, 5) dtype=float32, numpy=
array([[-0.4728725 ,  0.9244702 ,  0.3920691 , -0.7081981 ,  0.68729067]],
      dtype=float32)>, <tf.Variable 'sequential_23/dense_45/bias:0' shape=(5,) dtype=float32, numpy=array([0., 0., 0., 0., 0.], dtype=float32)>, <tf.Variable 'sequential_23/dense_46/kernel:0' shape=(5, 3) dtype=float32, numpy=
array([[-0.15573496,  0.5542514 ,  0.6054701 ],
       [ 0.40539628, -0.05237812, -0.82890296],
       [-0.53846896,  0.00232846,  0.29455036],
       [ 0.67539793,  0.45827943, -0.18992335],
       [ 0.4051667 , -0.76457375,  0.5170613 ]], dtype=float32)>, <tf.Variable 'sequential_23/dense_46/bias:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)>]
[<tf.Tensor: id=26788, shape=(), dtype=float32, numpy=2.094474>, <tf.Tensor: id=26789, shape=(), dtype=float32, numpy=-0.9927707>, <tf.Tensor: id=26790, shape=(), dtype=float32, numpy=-0.50399506>, <tf.Tensor:

In [0]:
#%tensorboard --logdir /tmp/tutorial/

In [0]:
%tensorboard --logdir /tmp/tutorial/